# 1.1 Load libraries

Note: After resolving conflicts, runs the entire notebook to make sure none of the old code is broken.

In [21]:
import pandas as pd
import numpy as np
import os

!pip install cvxpy --upgrade
import cvxpy as cvx

# 1.2 Gather data from github, where the csv files have been uploaded.
The data gathered is from a 5 year window, 4th Oct 2016 to 4th Oct 2021.

In [22]:
!git clone https://github.com/coldsonata/bt4016-project.git

fatal: destination path 'bt4016-project' already exists and is not an empty directory.


# 1.3 Load the stocks into memory and perform some preprocessing steps

The data is stored as key-value pairs in a dictionary, with the ticker referencing the respective dataframe. Following that, the date is changed into a datetime object and a new column for log-returns is added.

In [23]:
# load stocks into a dictionary
stocks = {}
do_not_append = ['FSELX','IPHI'] # FSELX is our fund, while IPHI does not have data and has been bought over by MRVL
for file_name in os.listdir('/content/bt4016-project/data/'):
  ticker = file_name.split(".")[0]
  if ticker in do_not_append:
    pass
  else:
    stocks[f'{ticker}'] = pd.read_csv(f'/content/bt4016-project/data/{file_name}') # for each stock, add its ticker as the key and dataframe as the value

In [24]:
print(list(stocks.keys())) # print the stock tickers for easy referencing

['AMD', 'TTMI', 'IIVI', 'JBL', 'AMBA', 'XLNX', 'AMAT', 'ENPH', 'TXN', 'AOSL', 'AVGO', 'OLED', 'CRUS', 'MCHP', 'DIOD', 'ASX', 'MTSI', 'MXL', 'SGH', 'MU', 'LRCX', 'QCOM', 'NVDA', '005930', 'INTC', 'TSM', 'MRVL', 'NXPI', 'SEDG', 'ON', 'SYNA', 'SMTC', 'ADI', 'AEIS', 'FLEX']


In [25]:
# Change date column into a datetime object
# Add a new column for log returns
for ticker, df in stocks.items():
  df['Date'] = df['Date'].apply(pd.to_datetime)
  df['LogReturns'] = np.append(np.nan,np.diff(np.log(df['Adj Close']))) # Fill dataframe with the log returns. The first value will be nan because there is no log returns for it.

In [26]:
# preview a stock
stocks['TXN'].head()

,Date,Open,High,Low,Close,Adj Close,Volume,LogReturns
0,2016-10-04,69.660004,69.910004,69.099998,69.589996,61.006310,4089500,NaN
1,2016-10-05,69.849998,70.839996,69.010002,70.599998,61.891724,4789600,0.014409
2,2016-10-06,70.440002,71.000000,70.029999,70.959999,62.207321,4061500,0.005086
3,2016-10-07,71.000000,71.059998,70.449997,70.870003,62.128426,3673600,-0.001269
4,2016-10-10,71.080002,71.330002,69.900002,69.930000,61.304375,4893200,-0.013352


In [27]:
top_30 = ['SYNA', 'MRVL', 'SGH', 'ENPH', 'JBL', 'CRUS', 'MCHP', 'AEIS', 'MTSI', 'AMAT', 'NVDA', 'ON', 'TTMI', 'SMTC', 'AVGO', 'XLNX', 'TSM', 'IIVI', 'QCOM', 'NXPI', 'FLEX', 'DIOD', 'AMD', 'MXL', 'ADI', 'INTC', 'TXN', 'LRCX', 'MU', 'OLED']
stocks_30 = {stock:df for stock,df in stocks.items() if stock in top_30}

# 1.4 Aggregate the log-returns into one dataframe

In [28]:
log_returns = pd.DataFrame(data=np.full((stocks['TXN'].shape[0],30),np.nan), columns = sorted(list(stocks_30.keys())), index = stocks_30['TXN'].Date) # create empty dataframe filled with NaNs, with index = Date and columns = stocks

# copy the log returns over from the stocks dictionary
for ticker, df in stocks_30.items():
  log_returns[ticker] = log_returns.index.map(stocks_30[ticker].set_index('Date')['LogReturns'])

# if simple returns are needed
simple_returns = np.exp(log_returns)-1
simple_returns['RF'] = np.repeat(0.0001,simple_returns.shape[0])

In [29]:
# preview log_returns
simple_returns.head()

,ADI,AEIS,AMAT,AMD,AVGO,CRUS,DIOD,ENPH,FLEX,IIVI,INTC,JBL,LRCX,MCHP,MRVL,MTSI,MU,MXL,NVDA,NXPI,OLED,ON,QCOM,SGH,SMTC,SYNA,TSM,TTMI,TXN,XLNX,RF
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2016-10-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0001
2016-10-05,0.006849,0.008323,0.011145,-0.027260,0.026205,0.018109,0.024402,-0.008333,0.030837,0.043100,0.011987,0.008280,0.014130,0.013046,0.011521,-0.001886,-0.005618,-0.016611,-0.001025,-0.007072,-0.006262,0.018062,0.002399,NaN,0.008727,0.049058,0.004607,0.019164,0.014513,0.006596,0.0001
2016-10-06,0.000928,0.027513,-0.001670,0.026549,0.004093,0.012919,0.003270,0.016807,0.004986,0.026619,0.002106,0.004562,0.041801,0.007405,0.000759,0.028106,0.001695,0.008687,-0.013044,0.009659,0.003819,0.002419,0.010473,NaN,-0.002884,0.054531,0.007861,0.001709,0.005099,0.000375,0.0001
2016-10-07,-0.004943,-0.015448,-0.004684,-0.030172,0.000115,-0.000739,-0.016760,-0.041322,-0.000709,-0.005418,0.000788,-0.010899,0.001597,-0.008309,-0.005311,-0.014013,-0.006768,0.000000,-0.007277,-0.004831,-0.000761,-0.012068,0.009624,NaN,-0.004700,0.017339,0.013325,0.014505,-0.001268,-0.010107,0.0001
2016-10-10,-0.016610,0.004184,-0.012437,0.013333,0.003214,0.023122,-0.007102,-0.086207,-0.004965,0.003891,-0.002099,0.002755,0.017937,-0.012085,0.004577,-0.010718,-0.006246,0.023445,0.003740,-0.001554,0.013707,-0.012215,-0.013785,NaN,-0.027243,0.014630,0.003207,0.000000,-0.013264,-0.012857,0.0001


The stock SGH has a lot of NaN values. We take a closer look at it.

In [30]:
# preview SGH
stocks['SGH'].head()

,Date,Open,High,Low,Close,Adj Close,Volume,LogReturns
0,2017-05-24,12.00,14.000,11.50,13.45,13.45,1415200,NaN
1,2017-05-25,13.85,14.320,13.35,13.50,13.50,237900,0.003711
2,2017-05-26,13.36,13.624,12.46,13.23,13.23,56200,-0.020203
3,2017-05-30,13.20,13.255,12.28,12.98,12.98,71200,-0.019077
4,2017-05-31,13.02,13.500,12.93,13.10,13.10,55700,0.009203


SGH only begins at a later date, explaining the NaNs we see in our earlier preview.

## 2.1 Optimization

In [31]:
# drop nans
simple_returns = simple_returns.dropna()

# get in-sample time window to perform optimization on (inclusive)
is_start_date = '2019-1-1'
is_end_date = '2020-12-31'

# get out-of-sample time window for evaluation (inclusive)
oos_start_date = '2020-1-1'
oos_end_date = '2021-12-31'

in_sample_window = simple_returns[(simple_returns.index>=is_start_date)&(simple_returns.index<=is_end_date)].copy()

### Try the pypfopt package, which makes it easier to add in L2 regularization so that we have less 0 weights.

In [32]:
!pip install PyPortfolioOpt --upgrade
from pypfopt import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt import objective_functions

     |████████████████████████████████| 60 kB 3.4 MB/s 


In [33]:
# Calculate expected returns and sample covariance using PyPortfolioOpt’s built-in methods
mu = expected_returns.mean_historical_return(in_sample_window, returns_data=True)
S = risk_models.sample_cov(in_sample_window, returns_data=True)

#print('Expected Returns:')
#print(mu)
#print('\n')
#print('Covariance Matrix:')
#print(S)

**From the pyportfolio webpage**

*In order to coerce the mean-variance optimizer to produce more non-negligible weights, we add what can be thought of as a “small weights penalty” to all of the objective functions, parameterised by γ (gamma).*

It is refered to as L2 regularisation despite having a different purpose from its usual use, to keep weights small. Here, is it used to keep weights bigger.

Note: Check different gamma values and portfolio's overall performance.

https://pyportfolioopt.readthedocs.io/en/latest/MeanVariance.html#pypfopt.objective_functions.L2_reg

In [34]:
# Set each stock to a sub-sector 
# Until someone has a better idea, use:
semiconductors = ['ADI','AMAT','AMD','AVGO','CRUS',
                  'DIOD','INTC','LRCX','MCHP','MRVL',
                  'MTSI','MU','MXL','NVDA','NXPI',
                  'OLED','ON','QCOM','SGH','SMTC',
                  'SYNA','TSM','TXN','XLNX']
others = ['AEIS','ENPH','FLEX','JBL','TTMI', 
               'IIVI','RF'] # AEIS and IIVI are also in the semiconductor subsector, but just putting them here first since they're in both areas. Every stock here except for IIVI are in the electrical/electronics scene. RF = risk-free

**Calculating risk aversion**

$U=E(R)-k*std(R)^2$

Assuming a portfolio consisting only of the market portfolio + risk-free assets, and expressing $E(R)$ and $std(R)$ in terms of their weights, $w$, we have:

$U=0.1*w+0.01*(1-w) - k*(w*0.18)^2$

where $w$ is the weight allocated to the market portfolio. Then,

$dU/dw = -0.0648*w*k+0.099$

If we choose w = 1, and then solving for $dU/dw = 0$, we get $k=1.528$

In [35]:
ef = EfficientFrontier(mu, S)
k = 1.528 
print(f"k = {k}")

ef.add_constraint(lambda x: x >= 0) # weights >= 0
ef.add_constraint(lambda x: x <= 0.25) # weight <= 0.25

sector_mapper = {} # initialise sector_mapper
for stock in semiconductors:
  sector_mapper[stock] = 'semiconductors'
for stock in others:
  sector_mapper[stock] = 'others'

sector_lower = {'others': 0.3} # at least 30% to others

ef.add_sector_constraints(sector_mapper=sector_mapper, sector_lower=sector_lower, sector_upper={}) # add sector constraint

ef.add_objective(objective_functions.L2_reg, gamma=5) # L2 regularisation parameter, defaults to 1. This adds a small weights penalty, gamma, to the objective function. Increase for more non-negligible weights. 

optimal_weights_portfolio = ef.max_quadratic_utility(risk_aversion = 2*k) # optimize by the quadratic utility function mean - k/2 * variance (hence the 2*k in the parameters)
ef.portfolio_performance(verbose=True, risk_free_rate=0.01) 
results = pd.Series(np.round(np.array(list(optimal_weights_portfolio.values())),4),index = in_sample_window.columns)
results

k = 2.777777777777778
Expected annual return: 179.1%
Annual volatility: 42.1%
Sharpe Ratio: 4.23


ADI     0.0111
AEIS    0.0047
AMAT    0.0264
AMD     0.0856
AVGO    0.0131
CRUS    0.0243
DIOD    0.0242
ENPH    0.2500
FLEX    0.0180
IIVI    0.0213
INTC    0.0000
JBL     0.0075
LRCX    0.0466
MCHP    0.0032
MRVL    0.0472
MTSI    0.0527
MU      0.0179
MXL     0.0042
NVDA    0.0602
NXPI    0.0143
OLED    0.0254
ON      0.0000
QCOM    0.0478
SGH     0.0000
SMTC    0.0000
SYNA    0.0375
TSM     0.0689
TTMI    0.0000
TXN     0.0219
XLNX    0.0138
RF      0.0525
dtype: float64

## Using CVX package (Just to check, drop this portion from the final submission)

Note: Not sure why the gamma here needs to be much smaller despite using the same function as above.

In [36]:
#returns_df = in_sample_window[semiconductors + others] # re-arrange dataframe with semiconductors in front and others behind for easy weight assignment 


In [37]:
# function stolen from tutorial 2
# modified to use the portfolio_return - k * portfolio_variance utility function
# TO-DO - OPTIMIZE BY SUB-SECTOR
# TO-DO - FIGURE OUT HOW TO ADD L2 REGULARIZATION WITHOUT RELYING ON PYPFOPT

def get_optimized_portfolio(returns_df, k):
    """
    Function that takes in the returns series of assets, minimizes the utility function, 
    and returns the portfolio weights
    
    Parameters
    ----------
    returns_df : pd.dataframe
        Dataframe containing log asset return series in each column
    
    k : float
        Risk-aversion
        
    Returns
    -------
    x : np.ndarray
        A numpy ndarray containing the weights of the assets in the optimized portfolio
    """
    
    returns = returns_df.T.to_numpy() # convert returns dataframe to numpy array 
    m = returns.shape[0] # m is the number of assets
  
    cov = np.cov(returns) # covariance matrix of returns
    
    x = cvx.Variable(len(semiconductors)) # creating variable of weights specific to semiconductors
    y = cvx.Variable(len(others)) # creating variable of weights specific to others
    w = cvx.hstack([x,y]) # stack the weights together so we can reference them as a whole
    
    portfolio_variance = cvx.quad_form(w, cov) # portfolio variance, in quadratic form, scaled by overall portfolio weight
   
    log_returns_df = np.log(returns_df+1)
  
    total_return_log = log_returns_df.sum().to_numpy() #this is in log space, change to simple return

    total_simple_return = np.exp(total_return_log) -1
    
    horizon_length = returns.shape[1]
    expected_mean = (1 + total_simple_return) ** (1 / horizon_length) - 1
   
    portfolio_return = sum(cvx.multiply(expected_mean, w)) # element wise multiplication, followed up by sum of weights
    
    # Objective Function                                                    
    # We want to minimize variance and maximize returns.                    
    #objective = cvx.Maximize(portfolio_return - k * portfolio_variance)  
    objectives = objective_functions.quadratic_utility(w, expected_mean, cov, 2*k) # multiply by 2 because pypfopt uses k/2
    objectives += objective_functions.L2_reg(w, gamma=0.01)
    objectives = cvx.Minimize(objectives)


    # Constraints
    # long only, sum of weights equal to 1, no allocation to a single stock great than 25% of portfolio, at least 30% to non-semiconductor stocks
    constraints = [w >= 0, sum(w) == 1, w <= 0.25, sum(y) >= 0.3]

    # use cvxpy to solve the objective
    problem = cvx.Problem(objectives, constraints)
    # retrieve the weights of the optimized portfolio
    result = problem.solve(solver="CVXOPT")

    print(f"Portfolio Returns: {np.round((portfolio_return.value+1)**252-1,3)*100}%, Portfolio Volatility: {np.round((252**0.5)*(portfolio_variance.value**0.5),3)*100}%") # print annualized results
    
    return w.value

In [38]:
#k = 1.528
#print(f"k = {k}")
#w = get_optimized_portfolio(returns_df, k=k)
#results = pd.Series(np.round(w,2), index = returns_df.columns) 
#results

## 2.2 Evaluation

Even if we don't need to compare old vs new portfolio, we still need this data for VaR calculations

In [39]:
ef.portfolio_performance(verbose=True, risk_free_rate=0.01) 
results

Expected annual return: 179.1%
Annual volatility: 42.1%
Sharpe Ratio: 4.23


ADI     0.0111
AEIS    0.0047
AMAT    0.0264
AMD     0.0856
AVGO    0.0131
CRUS    0.0243
DIOD    0.0242
ENPH    0.2500
FLEX    0.0180
IIVI    0.0213
INTC    0.0000
JBL     0.0075
LRCX    0.0466
MCHP    0.0032
MRVL    0.0472
MTSI    0.0527
MU      0.0179
MXL     0.0042
NVDA    0.0602
NXPI    0.0143
OLED    0.0254
ON      0.0000
QCOM    0.0478
SGH     0.0000
SMTC    0.0000
SYNA    0.0375
TSM     0.0689
TTMI    0.0000
TXN     0.0219
XLNX    0.0138
RF      0.0525
dtype: float64

In [40]:
original_portfolio = pd.read_csv('/content/bt4016-project/original_weights_best.csv')
#original_portfolio['Security\'s Percentage of the Total Net Assets'] = np.round(original_portfolio['Security\'s Percentage of the Total Net Assets'],3)/100
original_portfolio.index = original_portfolio['Ticker Symbol Given by the Exchange']
del original_portfolio['Ticker Symbol Given by the Exchange']
original_portfolio

,Security's Percentage of the Total Net Assets,Number of the Security's Shares,Market Value of the Security as of eff_dt,Name of the Security Held
Ticker Symbol Given by the Exchange,,,,
NVDA,19.019989,1891192.00,9.875805e+08,NVIDIA CORP
QCOM,6.719997,2288988.00,3.487044e+08,QUALCOMM INC
NXPI,6.079998,1983488.00,3.153944e+08,N X P SEMICONDUCTORS N V
MU,5.820000,4015660.00,3.018973e+08,MICRON TECHNOLOGY INC
MRVL,5.369999,5868093.00,2.789691e+08,MARVELL TECHNOLOGY GROUP LTD
AMD,5.289997,2994500.00,2.746256e+08,ADVANCED MICRO DEVICES INC
MCHP,4.779999,1794837.00,2.478849e+08,MICROCHIP TECHNOLOGY INC
AVGO,4.689999,555550.00,2.432476e+08,BROADCOM INC
XLNX,4.639999,1699866.00,2.409900e+08,XILINX INC


In [41]:
original_portfolio['Security\'s Percentage of the Total Net Assets'].sum() # missing 0.25% from somewhere

99.74995833300001